In [1]:
from datamodule.datamodule import *
from distributions.distributions import *
from utils import *
# MODEL:
from model.deepGARv1 import DeepGAR
# PYTORCH LIGHTNING:
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
#

In [2]:

name = 'electric'  # electric
train_size = 0.7
validation_size = 0.15
test_size = 0.15
batch_size = 64
custom_datamodule = DataModule(name, (train_size, validation_size, test_size), batch_size=batch_size)

LD2011_2014.txt.zip: 261MB [00:31, 8.26MB/s]                               


In [3]:
train_window = 24
test_window = 24
test_horizon = 24
test_stride = 24 # should be equal to the time horizon
test_delay = 0
train_loader, val_loader, test_loader = custom_datamodule.get_all(
    window=train_window,
    test_window=test_window,
    test_horizon=test_horizon,
    test_stride=test_stride,
    test_delay=test_delay
)

In [4]:
input_size = custom_datamodule.get_channels()
n_nodes = custom_datamodule.get_number_of_nodes()

distribution = GaussianDistribution()  # or StudentTDistribution()

test_loss = 'rmse'  # or 'mae'

perform_scaling = False

In [5]:
def run_model(run, train=False):

    model = DeepGAR(
        input_size=input_size,
        n_nodes=n_nodes,
        distribution=distribution,
        test_loss=test_loss,
        perform_scaling=perform_scaling
    )
    print(model)
    print_model_size(model)

    early_stopping_callback = EarlyStopping(
        monitor='val_loss',
        min_delta=0.00,
        patience=20,
        mode='min'
    )
    checkpoint_callback = ModelCheckpoint(
        dirpath=f'electricity_logs_{run}',
        save_top_k=1,
        monitor='val_loss',
        mode='min',
    )
    trainer = pl.Trainer(
        max_epochs=400,
        accelerator='cuda' if torch.cuda.is_available() else 'cpu',
        devices = 1,
        callbacks=[checkpoint_callback, early_stopping_callback]
    )
    if train:
        trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)
    # delete all
    try:
        del model
        del early_stopping_callback
        del checkpoint_callback
        del trainer
    except:
        pass


# list_available_models()

In [9]:
train_n_times=1
for i in range(train_n_times):
    run_model(i, train=True)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/marziolunghi/Documents/GitHub/DeepGAR/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:200: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

  | Name                  | Type              | Params
------------------------------------------------------------
0 | train_loss_fn         | NLL               | 0     
1 | test_loss_fn          | RMSE              | 0     
2 | distribution_sigma    | Softplus          | 0     
3 | encoder               | Linear            | 64    
4 | node_embeddings       | NodeEmbedding     | 11.8 K
5 | time                  | MultiLSTMCell     | 3.1 M 
6 | space_time            | GraphConvLSTMCell | 16.5 K
7 | distribution_mu       | Linear            | 33    
8 | distribution_presigma | Linear

DeepGAR(
  (train_loss_fn): NLL()
  (test_loss_fn): RMSE()
  (distribution_sigma): Softplus(beta=1, threshold=20)
  (encoder): Linear(in_features=1, out_features=32, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=370, embedding_size=32)
  (time): MultiLSTMCell(hidden_size=32)
  (space_time): GraphConvLSTMCell(hidden_size=32)
  (distribution_mu): Linear(in_features=32, out_features=1, bias=True)
  (distribution_presigma): Linear(in_features=32, out_features=1, bias=True)
)
Number of model (DeepGAR) parameters:   3106882


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/marziolunghi/Documents/GitHub/DeepGAR/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [6]:
# TESTING
early_stopping_callback = EarlyStopping(
        monitor='val_loss',
        min_delta=0.00,
        patience=20,
        mode='min'
    )
checkpoint_callback = ModelCheckpoint(
        dirpath=f'electricity_logs_{0}',
        save_top_k=1,
        monitor='val_loss',
        mode='min',
    )
trainer = pl.Trainer(
        max_epochs=400,
        accelerator='cuda' if torch.cuda.is_available() else 'cpu',
        devices = 1,
        callbacks=[checkpoint_callback, early_stopping_callback]
    )

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/marziolunghi/Documents/GitHub/DeepGAR/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:200: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [7]:
best_model_path = 'electricity_logs_0/epoch=1-step=164.ckpt'


best_model = DeepGAR.load_from_checkpoint(
    checkpoint_path=best_model_path
)
best_model.eval()
res = trainer.predict(model=best_model, dataloaders=test_loader)

Predicting: 0it [00:00, ?it/s]

In [8]:
from utils import get_metrics
get_metrics(res)

TypeError: squeeze() got an unexpected keyword argument 'dim'